# Exercise 3.1: Mastering filtering for Polars data frames

<hr>

Polars can be a bit frustrating during your first experiences with it. In this and the next few exercises, we will do our first practice with it. Stick with it! The more and more you use it, the more distant the memory of life without it will become.

We will work with a data set from [Kleinteich and Gorb, *Sci. Rep.*, **4**, 5355, 2014](https://doi.org/10.1038/srep05225), and was [featured in the New York Times](http://www.nytimes.com/2014/08/25/science/a-frog-thats-a-living-breathing-pac-man.html). They measured several properties about the tongue strikes of horned frogs. Let's take a look at the data set, which is in the file `~/git/data/frog_tongue_adhesion.csv`.

In [1]:
!head -20 data/frog_tongue_adhesion.csv

# These data are from the paper,
#   Kleinteich and Gorb, Sci. Rep., 4, 5225, 2014.
# It was featured in the New York Times.
#    http://www.nytimes.com/2014/08/25/science/a-frog-thats-a-living-breathing-pac-man.html
#
# The authors included the data in their supplemental information.
#
# Importantly, the ID refers to the identifites of the frogs they tested.
#   I:   adult, 63 mm snout-vent-length (SVL) and 63.1 g body weight,
#        Ceratophrys cranwelli crossed with Ceratophrys cornuta
#   II:  adult, 70 mm SVL and 72.7 g body weight,
#        Ceratophrys cranwelli crossed with Ceratophrys cornuta
#   III: juvenile, 28 mm SVL and 12.7 g body weight, Ceratophrys cranwelli
#   IV:  juvenile, 31 mm SVL and 12.7 g body weight, Ceratophrys cranwelli
date,ID,trial number,impact force (mN),impact time (ms),impact force / body weight,adhesive force (mN),time frog pulls on target (ms),adhesive force / body weight,adhesive impulse (N-s),total contact area (mm2),contact area without mucus (m

The first lines all begin with `#` signs, signifying that they are comments and not data. They do give important information, though, such as the meaning of the ID data. The ID refers to which specific frog was tested.

Immediately after the comments, we have a row of comma-separated headers. This row sets the number of columns in this data set and labels the meaning of the columns. So, we see that the first column is the date of the experiment, the second column is the ID of the frog, the third is the trial number, and so on.

After this row, each row represents a single experiment where the frog struck the target. So, these data are already in tidy format. 

**a)** Load in the data set into a data frame. Be sure to use the appropriate value for the `comment_prefix` keyword argument of `pl.read_csv()`.

**b)** Extract the impact time of all impacts that had an adhesive strength of magnitude greater than 2000 Pa. *Note*: The data in the `'adhesive strength (Pa)'` column is all negative. This is because the adhesive force is defined to be negative in the measurement. Without changing the data in the data frame, how can you check that the magnitude (the absolute value) is greater than 2000?

**c)** Extract the impact force and adhesive force for all of Frog II's strikes.

**d)** Extract the adhesive force and the time the frog pulls on the target for juvenile frogs (Frogs III and IV). *Hint*: We saw the `&` operator for Boolean indexing across more than one column. The `|` operator signifies OR, and works analogously. For technical reasons that we can discuss if you like, the Python operators `and` and `or` will **not** work for Boolean indexing of data frames.

<br />

## Solution

<hr>

In [2]:
import numpy as np
import polars as pl
import polars.selectors as cs

To read in the data frame, we use the `comment='#'` kwarg.

In [3]:
df = pl.read_csv('data/frog_tongue_adhesion.csv', comment_prefix='#')

# Take a look
df.head()

date,ID,trial number,impact force (mN),impact time (ms),impact force / body weight,adhesive force (mN),time frog pulls on target (ms),adhesive force / body weight,adhesive impulse (N-s),total contact area (mm2),contact area without mucus (mm2),contact area with mucus / contact area without mucus,contact pressure (Pa),adhesive strength (Pa)
str,str,i64,i64,i64,f64,i64,i64,f64,f64,i64,i64,f64,i64,i64
"""2013_02_26""","""I""",3,1205,46,1.95,-785,884,1.27,-0.29,387,70,0.82,3117,-2030
"""2013_02_26""","""I""",4,2527,44,4.08,-983,248,1.59,-0.181,101,94,0.07,24923,-9695
"""2013_03_01""","""I""",1,1745,34,2.82,-850,211,1.37,-0.157,83,79,0.05,21020,-10239
"""2013_03_01""","""I""",2,1556,41,2.51,-455,1025,0.74,-0.17,330,158,0.52,4718,-1381
"""2013_03_01""","""I""",3,493,36,0.8,-974,499,1.57,-0.423,245,216,0.12,2012,-3975


**b)** To extract the entries with strong adhesive strength, we need to use the `np.abs()` function to esure that the absolute value of the adhesive strength is above 2000.

In [4]:
df.filter(pl.col('adhesive strength (Pa)').abs() > 2000)['impact time (ms)']

impact time (ms)
i64
46
44
34
36
46
…
6
31
34


In [5]:
# c) Impact force and adhesive force for Frog II
df.filter(pl.col('ID')=='II')[['impact force (mN)', 'adhesive force (mN)']]

impact force (mN),adhesive force (mN)
i64,i64
1612,-655
605,-292
327,-246
946,-245
541,-553
…,…
515,-599
435,-364
383,-469


In [6]:
# d) Adhesive force and time frog pulls for frogs III and IV
(
    df
    .filter(pl.col('ID').is_in(['III', 'IV']))
    [["adhesive force (mN)", "time frog pulls on target (ms)"]]
)

adhesive force (mN),time frog pulls on target (ms)
i64,i64
-94,683
-163,245
-172,619
-225,1823
-301,918
…,…
-302,986
-216,1627
-163,2021


## Computing environment

In [7]:
%load_ext watermark
%watermark -v -p numpy,polars,jupyterlab

Python implementation: CPython
Python version       : 3.13.5
IPython version      : 9.4.0

numpy     : 2.2.6
polars    : 1.31.0
jupyterlab: 4.4.5

